### Grid Search using Rk4, also using the reduced S-B1 frame 

In [1]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
# import joptimise
using Printf
using DataFrames
using JSON
using CSV
plotly()

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err = ArgumentError("Package PlotlyBase not found in current path:\n- Run `import Pkg; Pkg.add(\"PlotlyBase\")` to install the PlotlyBase package.\n")
└ @ Plots C:\Users\yujit\.julia\packages\Plots\gzYVM\src\backends.jl:420


Plots.PlotlyBackend()

In [12]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
include("../src/SailorMoon.jl")   # relative path to main file of module


Main.SailorMoon

In [13]:
#### CALLBACK FUNCTIONS ####

# store the apoapsis value
function apoapsis_cond(u,t,int)
    # condition 1: SC is sufficiently far from the moon
    r = sqrt((u[1] - (1-param3b.mu2))^2 + u[2]^2 + u[3]^2) # SC-Moon distance
    moon_soi = 5000 / param3b.lstar # define "sphere of influence"
    
    if sqrt(u[1]^2 + u[2]^2 + u[3]^2) > 2.0
        # condition 2: dot product of velocity and position is zero
        return dot((u[1:3] + [param3b.mu2, 0.0, 0.0]), u[4:6])
    else
        return NaN
    end
end

# store the periapsis value and terminate
function periapsis_cond(u,t,int)
    r = sqrt((u[1] + param3b.mu2)^2 + u[2]^2 + u[3]^2)  # SC-earth distance
    earth_leo_ub = 10000 / param3b.lstar  # km
    earth_leo_lb = 200  / param3b.lstar  # km
    
    if earth_leo_lb < r < earth_leo_ub
        return dot((u[1:3] + [param3b.mu2, 0.0, 0.0]), u[4:6])
    else 
        return NaN
    end
end

terminate_affect!() = true
no_affect!() = false

# include callback functions 
apoapsis_cb  = ContinuousCallback(apoapsis_cond, terminate_affect!)
periapsis_cb = ContinuousCallback(periapsis_cond, terminate_affect!)
cbs = [apoapsis_cb, periapsis_cb];



In [14]:
function rhs_bcr4bp_thrust!(du,u,p,t)
    # unpack arguments
    μ, μ_3, t0, a_s, ω_s, τ, θ, β, mdot, tmax = p
    # decompose state
    x, y, z, vx, vy, vz, mass = u

    # calculate radii
    r1 = sqrt( (x+μ)^2 + y^2 + z^2 )
    r2 = sqrt( (x-1+μ)^2 + y^2 + z^2 )

    # sun position
    xs = a_s*cos(ω_s*t + t0)
    ys = a_s*sin(ω_s*t + t0)
    zs = 0.0
    r3 = sqrt( (x-xs)^2 + (y-ys)^2 + (z-zs)^2 )

    # compute delta-V vector
#     dir_v = dv_lvlh2inertial(u[1:6], [τ, θ, β])
    dir_v = [0.0, 0.0, 0.0]
    ts = tmax*dir_v

    # position-state derivative
    du[1] = vx
    du[2] = vy
    du[3] = vz

    # velocity derivatives: Earth, moon, sun
    du[4] =  2*vy + x - ((1-μ)/r1^3)*(μ+x)  + (μ/r2^3)*(1-μ-x)  + ( -(μ_3/r3^3)*(x-xs) - (μ_3/a_s^3)*xs ) + ts[1]/mass
    du[5] = -2*vx + y - ((1-μ)/r1^3)*y      - (μ/r2^3)*y        + ( -(μ_3/r3^3)*(y-ys) - (μ_3/a_s^3)*ys ) + ts[2]/mass
    du[6] =           - ((1-μ)/r1^3)*z      - (μ/r2^3)*z        + ( -(μ_3/r3^3)*(z)    - (μ_3/a_s^3)*zs ) + ts[3]/mass
    
#     println(r1, " ", r2, " ", r3)
#     println(((1-μ)/r1^3)*(μ+x), " ", (μ/r2^3)*(1-μ-x), " ",  -(μ_3/r3^3)*(x-xs) )

    # mass derivative
    du[7] = -mdot*τ
end

rhs_bcr4bp_thrust! (generic function with 1 method)

In [15]:
function plot_circle(radius, x, y, n=50)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [17]:
param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

## set up of initial condition (Lyapunov orbit)
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1) # monodromy eigenvector

## Grid search parameters: CHANGE HERE
n = 10
ϕ_vec    = [0.0] #LinRange(0, 2*pi, n+1)[1:n] #
epsr_vec = [1e-6]   #10 .^(-12:-6)
epsv_vec = [1e-6]   #10 .^(-12:-6)
θs_vec    = [0.0]  #LinRange(0, 2*pi, n+1)[1:n]  # [2.890265, 3.015929]     
tof_bck  = 120 * 86400 / param3b.tstar


Halo guess Az_km: 1200.0
Linear stability ν = 618.7618470919092


27.59646130590521

In [28]:

## make initial conditions 
grids = []
for ϕ0 in ϕ_vec
    for ϵr in epsr_vec
        for ϵv in epsv_vec
            for θsf in θs_vec
                
                θmf = π - θsf                
                # arrival LPO object
                LPOArrival = SailorMoon.CR3BPLPO2(
                    res.x0, res.period, ys0, prob_cr3bp_stm, ϵr, ϵv, Tsit5(), 1e-12, 1e-12
                );
                
                xf = SailorMoon.set_terminal_state2(ϕ0, θmf, param3b, LPOArrival)
                # in Sun-B1 frame
                println(param3b.oms)
                xf_sb1 = vcat(SailorMoon.transform_EMrot_to_SunB1(xf, θsf, param3b.oms), 1.0)
                
                push!(grids, [ϕ0, ϵr, ϵv, θsf, xf_sb1])
                println(xf)
                
            end
        end
    end
end

# include callback functions 
apoapsis_cb  = ContinuousCallback(apoapsis_cond, terminate_affect!)
periapsis_cb = ContinuousCallback(periapsis_cond, terminate_affect!)
cbs = [apoapsis_cb, periapsis_cb];

-0.9251999994040079


LoadError: UndefVarError: param3b not defined

In [25]:
svf_ = zeros(Float64, 1, 7)
params = [param3b.mu2, param3b.mus, param3b.as, 0.0, param3b.oms, param3b.omb, 0.0, 0.0, 0.0, 0.0, 0.0]
tspan = [0, -tof_bck]

prob = ODEProblem(rhs_bcr4bp_sb1frame2!, svf_, tspan, params)
sol_bck = solve(prob_bck, Tsit5(), reltol=1e-12, abstol=1e-12);


LoadError: UndefVarError: rhs_bcr4bp_sb1frame2! not defined

In [20]:

## make ensemble problems
function prob_func(prob, i, repeat)
    print("\rproblem # $i")
    remake(prob, u0=grids[i][5], p=[param3b.mu2, param3b.mus, param3b.as, grids[i][4], param3b.oms, param3b.omb,
            0.0, 0.0, 0.0, 0.0, 0.0])
end

ensemble_prob = EnsembleProblem(prob, prob_func=prob_func)
sim = solve(ensemble_prob, Tsit5(), EnsembleThreads(), trajectories=length(grids),
            callback=cbs, reltol=1e-12, abstol=1e-12,
            save_everystep=true);


## data extraction and make csv
# make dataframe
entries = ["id", "phi0", "epsr", "epsv", "thetaf", "ra", "rp", "dt1", "dt2", "x_ra", "x_rp", "tof", "lfb"]
df = DataFrame([ name =>[] for name in entries])

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)

id = 1
solution = []
# extract the ensemble simulation
for (idx,sol) in enumerate(sim)
    global lfb_count = 0
    
    ϕ0 = grids[idx][1]
    ϵr = grids[idx][2]
    ϵv = grids[idx][3]
    θsf = grids[idx][4]
    plot!(pcart, Array(sol)[1,:], Array(sol)[2,:], color=:blue, linewidth=1.0, label="sol $idx", linestyle=:solid)
    global solution = sol
    
    if sol.retcode == :Terminated
#         println(periapsis_cond(sol.u[end], 0.0, 0.0))
        if ~isnan(periapsis_cond(sol.u[end], 0.0, 0.0))  # == true
            t_vec = -sol.t[:]
            rp = sqrt(sol.u[end][1]^2 + sol.u[end][2]^2 + sol[end][3]^2)
            x_rp = sol.u[end]
            
            # find apoapsis
            r_vec = sqrt.((Array(sol)[1,:] .- [param3b.mu2]).^2+Array(sol)[2,:].^2+Array(sol)[3,:].^2)
            ra, id_ra = findmax(r_vec)
            x_ra = sol.u[id_ra]
            dt_ra = - sol.t[id_ra]
            dt_rp = -sol.t[end] - dt_ra 
            
            # flag: lunar flyby? 
            rm_vec = sqrt.((Array(sol)[1,:] .- [1-param3b.mu2]).^2+Array(sol)[2,:].^2+Array(sol)[3,:].^2)
            id_lfb = findall(rm_vec .< (66000 / param3b.lstar) .&& t_vec .> (10*86400/param3b.tstar))
            
            if ~isempty(id_lfb)
#                 println("there might be a perilune")
                for k in id_lfb
                    if ~isnan(perilune_cond(sol.u[k], sol.t[k], 0.0))
                        global lfb_count += 1
                    end
                end    
            end

            
            tof = -sol.t[end]
            push!(df, [id, ϕ0, ϵr, ϵv, θsf, rp, ra, dt_ra, dt_rp, x_ra, x_rp, tof, lfb_count])
            println("idx $idx is a success!")
            plot!(pcart, Array(sol)[1,:], Array(sol)[2,:], color=:blue, linewidth=1.0, label="sol $idx", linestyle=:solid)
#             println(sol.u, sol.t)
            global id += 1
        end
    end
end

# print(df)
# CSV.write("grid_search2.csv", df)



LoadError: UndefVarError: prob not defined

In [ ]:
println(solution[1])
plot2 = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)

plot!(plot2, Array(solution)[1,:], Array(solution)[2,:], color=:blue, linewidth=1.0, linestyle=:solid)
scatter!(plot2, [solution.u[1][1]], [solution.u[1][2]], color=:red, shape=:circle)

moon = plot_circle(1-param3b.mu2, param3b.as , 0.0)
earth = plot_circle(param3b.mu2, param3b.as, 0.0)
moon_soi_outer = plot_circle(1-param3b.mu2+66000/param3b.lstar, param3b.as, 0.0)

plot!(plot2, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(plot2, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(plot2, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")